# FBXL4

Variants in FBXL4 are associated with [Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)](https://omim.org/entry/605654)
This notebook collects clinical data from several publications.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from IPython.display import display, HTML
import pyphetools
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import CohortValidator
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.63


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel('input/FBXL4-data.xlsx')

In [4]:
df.head(2)

ID  omim_id  \
0  34738379_P1   615471   
1  32559514_P1   615471   

                                                          omim_title  \
0  MITOCHONDRIAL DNA DEPLETION SYNDROME 13 (ENCEPHALOMYOPATHIC TYPE)   
1  MITOCHONDRIAL DNA DEPLETION SYNDROME 13 (ENCEPHALOMYOPATHIC TYPE)   

                                                           omim_name  \
0  Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)   
1  Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)   

      hgnc_id gene_symbol               Zigosity  \
0  HGNC:13601       FBXL4  Compound heterozygous   
1  HGNC:13601       FBXL4             homozygous   

                        Location                          Variant annotation  \
0  NM_001278716.1/NM_001278716.1  c.772G>T(p.Gly258*)/c.1061G>C(p.Trp354Ser)   
1                 NM_001278716.1                     c.993dupA (p.L332Tfs*3)   

         Consequence  ... Gender Age at death  \
0  nonsense/missense  ...    NaN          NaN   
1                NaN  ...    NaN          NaN   

                                                                                                                                                                                                                                       Phenotype  \
0  abnormal eyebrow morphology HP:0000534 , triangular face HP:0000325, abnormal shape of the palpebral fissure HP:0200005, mandibular prognathia HP:0000303, increased serum lactate HP:0002151 HP:0002151, increased serum pyruvate HP:0003542   
1          global developmental delay HP:0001263, hypotonia HP:0001252, abnormal eating behavior HP:0100738, hypertelorism HP:0000316, short palpebral fissure HP:0012745, internuclear ophthalmoplegia HP:0030773, wide nasal bridge HP:0000431   

  Prenatal ultrasound phenotype                               MRI phenotype  \
0                           NaN                                         NaN   
1                           NaN  abnormality of brain morphology HP:0012443   

  Cardiac phenotype                  Family history  \
0               NaN  Primigravid, no family history   
1               NaN               no family history   

                                          Source           PMID  \
0  https://doi.org/10.24953/turkjped.2021.05.025  PMID:34738379   
1      https://doi.org/10.1016/j.jns.2020.116948  PMID:32559514   

                                                                                                   title  
0                                    Entero-encephalopathy due to FBXL4-related mtDNA depletion syndrome  
1  Novel homozygous mutation in the FBXL4 gene is associated with mitochondria DNA depletion syndrome-13  

[2 rows x 26 columns]

In [5]:
df['patient_id'] = df['ID']
df.set_index('ID', inplace=True)

<h1>Variants</h1>
<p>Variant data is available in strings such as c.772G>T(p.Gly258*)/c.1061G>C(p.Trp354Ser) or c.772G>T(p.Gly258*) for homozygous variants.
We need to extract the HGVS cDNA string to pass to Variant Validator</p>

In [6]:
def extract_cdna(variant):
    """
    split strings like c.772G>T(p.Gly258*) on the open-parenthesis symbol and return the first part
    """
    v = variant.split("(")[0]
    v = v.replace(" ", "").replace("p.","")
    return v
    
def extract_variant_1(variants):
    """
    Split on the slash ("/") and return the first part (or entire string for homozygous)
    """
    v1 = variants.split("/")[0]
    return extract_cdna(v1)

def extract_variant_2(variants):
    """
    Split on the slash ("/") and return the second part (or entire string for homozygous)
    """
    fields = variants.split("/")
    if len(fields) == 2:
        return extract_cdna(fields[1])
    else:
        # there was only one variant
        return extract_cdna(variants)

In [7]:
df["var1"] = df['Variant annotation'].apply(lambda x: extract_variant_1(x))
df["var2"] = df['Variant annotation'].apply(lambda x: extract_variant_2(x))

In [8]:
FBXL4_transcript = "NM_001278716.2"
vman = VariantManager(df=df, individual_column_name="patient_id", gene_symbol="FBXL4", allele_1_column_name="var1",
                      allele_2_column_name="var2", transcript=FBXL4_transcript)

<h2>Mapping the age column</h2>
<p>We add a dictionary to the corresponding ISO 8601 age strings. Anything not here will be mapped to "not provided"</p>

In [9]:
string_to_iso_dict = {
    "1.5": "P1Y6M",
    "3.5": "P3Y6M",
    'birth': "P1D",
    '51 days': "P1M21D",
    '19 months': "P1Y7M",
    '10 monhts': "P10M",
    '14 months': "P1Y2M",
    '23 months': "P1Y11M",
    '22 mohts': "P1Y10M",
    '9 months': "P9M",
    '12 years': "P12Y",
    '1 day of age': "P1D",
    '4.5 years': "P4Y6M",
    '4 months': "P4M",
    '1 month': "P1M"
}
ageMapper = AgeColumnMapper.custom_dictionary(column_name='Age of diagnosis', string_to_iso_d=string_to_iso_dict)
#ageMapper.preview_column(df['Age of diagnosis']).head()

In [10]:
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", unknown_symbol="nan", column_name="Gender")
#sexMapper.preview_column(df['Gender']).head()

In [11]:
aod_d = {
    "7 months": "P7M",
    "10 months": "P10M",
    "4 days": "P4D",
    "5 days": "P5D",
    "3 days": "P3D",
    "9 monhts": "P9M",
    "12 years": "P12Y",
    "3 years": "P3Y",
    "4 years": "P4Y",
    "2 years": "P2Y",
    "2.5 years": "P2Y6M",
    "16 months": "P1Y4M",
}
aodMapper = AgeOfDeathColumnMapper(column_name='Age at death', string_to_iso_d=aod_d)

<h1>Mapping phenotypic features</h1>
<p>We use the CustomColumnMapper class, which uses text mining to recognize one or more HPO terms in each table cell. The preview
column function can be used to check results, which appear very accurate. Each mapper is put into a map that is used to encode the entire cohort, below</p>

In [12]:
column_mapper_list = list()

In [13]:
phenotypeColumnMapper = OptionColumnMapper(column_name='Phenotype',concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(phenotypeColumnMapper)
phenotypeColumnMapper.preview_column(df)

mapping count
0                   Abnormal eyebrow morphology (HP:0000534) (observed)     1
1                               Triangular face (HP:0000325) (observed)     1
2       Abnormal shape of the palpebral fissure (HP:0200005) (observed)     1
3                         Mandibular prognathia (HP:0000303) (observed)     1
4   Increased circulating lactate concentration (HP:0002151) (observed)    18
..                                                                  ...   ...
91                    Abnormal pinna morphology (HP:0000377) (observed)     1
92                          Concave nasal ridge (HP:0011120) (observed)     1
93                               Microphthalmia (HP:0000568) (observed)     1
94                       Developmental cataract (HP:0000519) (observed)     1
95                         Horizontal nystagmus (HP:0000666) (observed)     1

[96 rows x 2 columns]

In [14]:
prenatalUSmapper =  OptionColumnMapper(column_name='Prenatal ultrasound phenotype',concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(prenatalUSmapper)
prenatalUSmapper.preview_column(df)

mapping count
0          Single umbilical artery (HP:0001195) (observed)     1
1    Enlarged fetal cisterna magna (HP:0011427) (observed)     1
2                    Hydrocephalus (HP:0000238) (observed)     1
3                   Polyhydramnios (HP:0001561) (observed)     1
4                 Ventriculomegaly (HP:0002119) (observed)     1
5  Intrauterine growth retardation (HP:0001511) (observed)     1
6      Mild fetal ventriculomegaly (HP:0010952) (observed)     1

In [15]:
mriMapper = OptionColumnMapper(column_name='MRI phenotype',concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(mriMapper)
mriMapper.preview_column(df)

mapping  \
0                        Abnormal brain morphology (HP:0012443) (observed)   
1                            Periventricular cysts (HP:0007109) (observed)   
2                              CNS hypomyelination (HP:0003429) (observed)   
3                             Global brain atrophy (HP:0002283) (observed)   
4                       Abnormal cortical gyration (HP:0002536) (observed)   
5                            Cerebellar hypoplasia (HP:0001321) (observed)   
6                                 Ventriculomegaly (HP:0002119) (observed)   
7                             Thin corpus callosum (HP:0033725) (observed)   
8   Hyperintensity of cerebral white matter on MRI (HP:0030890) (observed)   
9      Abnormal basal ganglia MRI signal intensity (HP:0012751) (observed)   
10                 Frontotemporal cerebral atrophy (HP:0006892) (observed)   
11                                Cerebral atrophy (HP:0002059) (observed)   
12                                Encephalomalacia (HP:0040197) (observed)   
13   Periventricular white matter hyperintensities (HP:0030891) (observed)   
14                                   Brain atrophy (HP:0012444) (observed)   
15                      Focal white matter lesions (HP:0007042) (observed)   
16               Abnormal basal ganglia morphology (HP:0002134) (observed)   
17                         Enlarged cisterna magna (HP:0002280) (observed)   
18                      Intracranial cystic lesion (HP:0010576) (observed)   

   count  
0      1  
1      2  
2      3  
3      1  
4      1  
5      1  
6      3  
7      3  
8      1  
9      1  
10     1  
11     2  
12     1  
13     1  
14     1  
15     6  
16     1  
17     2  
18     1

In [16]:
cardiacMapper = OptionColumnMapper(column_name='Cardiac phenotype',concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(cardiacMapper)
cardiacMapper.preview_column(df)

mapping count
0  Pulmonary arterial hypertension (HP:0002092) (observed)     1
1    Multifocal atrial tachycardia (HP:0011701) (observed)     1
2      Hypertrophic cardiomyopathy (HP:0001639) (observed)     1
3     Left ventricular hypertrophy (HP:0001712) (observed)     1

<h1>Putting it all together</h1>

In [17]:
mdds13 = Disease(disease_id="OMIM:615471", disease_label="Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)")
disease_d = {"615471": mdds13}
diseaseMapper = DiseaseIdColumnMapper(column_name="omim_id", disease_id_map=disease_d)

In [18]:
encoder = MixedCohortEncoder(df=df,
                            hpo_cr=hpo_cr,
                             column_mapper_list=column_mapper_list,
                             individual_column_name="patient_id",
                             disease_id_mapper=diseaseMapper,
                             pmid_column="PMID",
                             title_column="title",
                             sexmapper=sexMapper,
                             agemapper=ageMapper,
                             age_of_death_mapper=aodMapper,
                             metadata=metadata
                        )

AttributeError: 'HpoExactConceptRecognizer' object has no attribute 'get_hpo_ontology'

In [19]:
individuals = encoder.get_individuals()

In [20]:
# retrieve the variant strings and add Variant objects to each individual
# the individual id (i.id) is also the index of the pandas dataframe
variant_d = vman.get_variant_d()
for i in individuals:
    row = df.loc[i.id] 
    v1 = row['var1']
    v2 = row['var2']
    #print(f"{i.id}: v1={v1} and v2={v2}")
    if v1 == v2:
        var1 = variant_d.get(v1)
        var1.set_homozygous()
        i.add_variant(var1)
    else:
        var1 = variant_d.get(v1)
        var2 = variant_d.get(v2)
        var1.set_heterozygous()
        var2.set_heterozygous()
        i.add_variant(var1)
        i.add_variant(var2)

In [21]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

<h2>Visualization</h2>

In [22]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

In [23]:
MixedCohortEncoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 30 GA4GH phenopackets to the directory phenopackets
